# 第11章 TensorBoard可视化

- 为了更好地管理、调试和优化神经网络的训练过程，TensorFlow提供了一个可视化工具TensorBoard；TensorBoard可以有效地展示TensorFlow在运行过程中的计算图、各种指标随着时间的变化趋势以及训练中使用到的图像等信息

## 11.1 TensorBoard简介

- TensorBoard是TensorFlow的可视化工具，它可以通过TensorFlow程序运行过程中输出的日志文件可视化TensorFlow程序的运行状态；TensorBoard和TensorFlow跑在不同的进程中，TensorBoard会自动读取最新的TensorFlow日志文件，并呈现当前TensorFlow程序运行的最新状态

- 以下代码展示了一个简单的TensorFlow程序，在这个程序中完成了TensorBoard日志输出的功能

In [1]:
import tensorflow as tf

# 定义一个简单的计算图，实现向量加法的操作
input1 = tf.constant([1.0, 2.0, 3.0], name='input1')
input2 = tf.Variable(tf.random_uniform([3]), name="input2")
output = tf.add_n([input1, input2], name="add")

# 生成一个写日志的writer，并将当前的TensorFlow计算图写入日志；TensorFlow提供了多种写日志文件的API，在11.3节详细介绍
writer = tf.summary.FileWriter("./log", tf.get_default_graph())
writer.close()

Instructions for updating:
Colocations handled automatically by placer.


- 以上程序输出了TensorFlow计算图的信息，所以运行TensorBoard时，可以看到这个向量相加程序计算图可视化之后的结果

## 11.2 TensorFlow计算图可视化

### 11.2.1 命名空间与TensorBoard图上节点

- 为了更好地组织可视化效果图中的计算节点，TensorBoard支持通过TensorFlow命名空间来整理可视化效果图上的节点；在TensorBoard的默认视图中，TensorFlow计算图中同一个命名空间下的所有节点会被缩略成一个节点，只有顶层命名空间中的节点才会被显示在TensorBoard可视化效果图上
- 除了tf.variable_scope函数，tf.name_scope函数也提供了命名空间管理的功能，这两个函数在大部分情况下是等价的，唯一的区别是在使用tf.get_variable函数时，以下代码简单地说明了这两个函数的区别

In [2]:
import tensorflow as tf

with tf.variable_scope("foo"):
    # 在命名空间foo下获取变量"bar"，于是得到的变量名称为"foo/bar"
    a = tf.get_variable("bar", [1])
    print a.name # 输出：foo/bar:0
    
with tf.variable_scope("bar"):
    # 在命名空间bar下获取变量“bar”，于是得到的变量名称为“bar/bar”；此时变量"bar/bar"和变量“foo/bar”并不冲突，可以正常运行
    b = tf.get_variable("bar", [1])
    print b.name # 输出：bar/bar:0
    
with tf.name_scope("a"):
    # 使用tf.Variable函数生成变量会受tf.name_scope影响，于是这个变量的名称为“a/Variable”
    a = tf.Variable([1])
    print a.name # 输出：a/Variable:0
    
    # tf.get_variable函数不受tf.name_scope函数的影响，于是变量并不在a这个命名空间中
    a = tf.get_variable("b", [1])
    print a.name # 输出：b:0
    
with tf.name_scope("b"):
    # 因为tf.get_variable不受tf.name_scope影响，所以这里将试图获取名称为"a"的变量，然而这个变量已经被声明了，于是这里会报重复声明的错误
    tf.get_variable("b", [1])

foo/bar:0
bar/bar:0
a/Variable:0
b:0


ValueError: Variable b already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-2-ab98be4df6bf>", line 19, in <module>
    a = tf.get_variable("b", [1])
  File "/Library/Python/2.7/site-packages/IPython/core/interactiveshell.py", line 2878, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/Library/Python/2.7/site-packages/IPython/core/interactiveshell.py", line 2818, in run_ast_nodes
    if self.run_code(code, result):


In [2]:
# 通过对命名空间管理，改进11.1节中向量相加的样例代码
import tensorflow as tf

# 将输入定义放入各自的命名空间中，从而使得TensorBoard可以根据命名空间来整理可视化效果图上的节点
with tf.name_scope("input1"):
    input1 = tf.constant([1.0, 2.0, 3.0], name="input1")
with tf.name_scope("input2"):
    input2 = tf.Variable(tf.random_uniform([3]), name="input2")
output = tf.add_n([input1, input2], name="add")

writer = tf.summary.FileWriter("./log", tf.get_default_graph())
writer.close()

## 小结

- TensorBoard是TensorFlow自带的工具，不需要额外的安装过程；虽然二者运行在不同的进程中，但是TensorBoard会实时读取TensorFlow程序输出的日志文件从而获取最新的TensorFlow程序运行状态
- 通过TensorBoard，一方面可以更好地了解TensorFlow计算图的结构以及每个TensorFlow计算节点在运行时的时间以及内存消耗，另一方面也可以通过TensorBoard可视化神经网络模型训练过程中各种指标的变化趋势，直观地了解神经网络的训练情况